# Chapter 10-5. Advanced CNN (VGG)

- making VGG network

## VGG-net
Network made by Oxford VGG(Visual Geometry Group) </br>
- vgg11 ~ vgg19 까지 만들 수 있음. 크기 다양함.
- RGB 3x244x244 입력을 기준으로 제작됨.
- 특징: 3byte로 구성됨.

---

## 기본 VGG net 만들고 나의 VGG 생성하기

In [1]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

In [2]:
__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]


model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}

In [3]:
class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        
        self.features = features #convolution
        
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )# FC layer
        
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x) #Convolution 
        x = self.avgpool(x) # avgpool
        x = x.view(x.size(0), -1) # 일렬로 쌓음
        x = self.classifier(x) #FC layer
        return x

    def _initialize_weights(self):
        for m in self.modules(): # 만약 m이 Conv2d일 때,
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                # ㄴ 우리가 어떤 activation 함수를 쓰느냐에 따라서 weight의 값을 잘 초기화를 할 수도 있다.
                # 그에 맞춰서 
                if m.bias is not None: # bias값을 0으로 초기화한다.
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d): # BatchNorm일 경우에도 normalization
                nn.init.constant_(m.weight, 1) # 전부 1
                nn.init.constant_(m.bias, 0) # 전부 0
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

- __ init __ 함수는 features, num_classes, init_weights 세 개를 인자로 가짐.
- classifier - FC layer들.
- 가장 하단부에 FC layer 3개.
- forward: Convolution 진행 부분, avgpool 진행한 이후에 일렬로 펼치고, FC layer 통과하여 최종 결과 도출
- self.modules 했을 때는 features가 넘겨 줬던 레이어의 값을 하나씩 m에게 return. 
- m이 처음에는 convolution, convolution이 들어가게 되면 kaiming_normalization(필터값), nonlinearity는 relu라는 이야기 (다양한 activation function 집어넣을 수 있음.)

#### feature값을 어떻게 만들어서 넣어 줄 수 있느냐?가 중요한 부분
1. make_layers
2. cfg dictionary

In [4]:
# 'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
    
def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v # 중요
                     
    return nn.Sequential(*layers

하나의 Convolution layer 완성

make_layers function: model = make_later(cfg['A'], batch_norm=False)을 확인해 보겠다는 이야기.

for example) cfg == cfg['A'] 이고 v가 64일 때의 conv2d값 보기:  </br>
> conv2d = nn.Conv2d(3, 64, kernel_size=3, padding=1)

> list에 nn.ReLU(inplace=True) 추가

그 다음 v가 'M'일 때: </br>
> nn.MaxPool2d(kernel_size=2, stride=2)

그 다음 값 v가 128 </br>
> ###### in_channel 같은 경우에 64로 바뀌어서 64로 들어감. 

> conv2d = nn.Conv2d(64, 128, kernel_size=3, padding=1)

> else: layers += [conv2d, nn.ReLU(inplace=True)] 들어감

그 다음 v가 'M' </br>
> nn.MaxPool2d(kernel_size=2, stride=2)

그 다음 </br>
> conv2d = nn.Conv2d(128, 256, kernel_size=3, padding=1)

> nn.ReLu(inplace=True)

그 다음 </br>
> conv2d = nn.Conv2d(256, 256, kernel_size=3, padding=1)

> nn.ReLu(inplace=True)

그 다음 </br>
> nn.MaxPool2d(kernel_size=2, stride=2) </br>
conv2d = nn.Conv2d(256, 512, kernel_size=3, padding=1) </br>
nn.ReLu(inplace=True) </br>
conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) </br>
nn.ReLu(inplace=True) </br>
nn.MaxPool2d(kernel_size=2, stride=2) </br>
conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) </br>
nn.ReLu(inplace=True) </br>
conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) </br>
nn.ReLu(inplace=True) </br>
nn.MaxPool2d(kernel_size=2, stride=2)

만약 True라고 가정을 하면: layer list에 if문의 layer값이 추가가 된다. (Conv2d) </br>

만약 False라면: layer list에 else문의 layer값이 추가가 된다. (ReLU)

- Convolution Layer가 8개가 되는가?

In [16]:
conv = make_layers(cfg['A'])

In [17]:
conv

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): ReLU(inplace=True)
  (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (14): ReLU(inplace=True)
  (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (16): Conv2d(512, 512, kernel_size=(3, 3), stride=

- 위에서 이야기했던 logic가 똑같이 작동하는 것을 볼 수 있음

*in_channels = v*라고 하는 것이 중요
- 이유: nn.Conv2d 처리 후 채널 수가 64 채널로 변경됨.
- 다음번 채널에서 64 채널을 받아야 하기 때문에, in_channel이 64로 바뀜.
- 지금 layer의 값은 빈 레이어에 두 개가 들어갔기 때문에, nn.ReLU 값을 추가한 것이 layer

In [5]:
cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], #8 + 3 =11 == vgg11
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], # 10 + 3 = vgg 13
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'], #13 + 3 = vgg 16
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'], # 16 +3 =vgg 19
    'custom' : [64,64,64,'M',128,128,128,'M',256,256,256,'M']
}

cfg dictionary에는 A, B, D, E 들어가 있음 </br>
이것에 맞춰서 make_layers 동작해 보기(이에 input을 받는 것도 A, B, D, E)

In [20]:
conv = make_layers(cfg['custom'])

In [21]:
conv

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (10): ReLU(inplace=True)
  (11): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): ReLU(inplace=True)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): Conv2d(2

위 식에서 *batch_norm만 True*라고 설정 바꾸어 주기

In [18]:
conv = make_layers(cfg['custom'], batch_norm=True)

In [19]:
conv

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace=True)
  (13): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (14): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats

Conv2d + Batch 추가되어 있는 것 확인 가능

기존에 있는 torch vision에 있는 VGG를 내가 원하는 상태로 수정하여 사용 가능
- 여기까지는 make_layer을 통해 layer을 쌓은 시퀀스를 출력하는 형태 본 것

#### 실재로 완성된 VGG는?

(1) features라고 불리는 convolution layer을 make_layers 가지고 와서 만들면 됨. </br> 
     - 내가 만든 걸 넣고 싶다면 'custom' </br>
(2) class가 10개이기 때문에 num_classes 10개 넣어 줌 </br>
(3) init_weight같은 경우 True로 넣어 줌 </br>

In [22]:
CNN = VGG(make_layers(cfg['custom']), num_classes=10, init_weights=True)

In [8]:
CNN

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), paddin

#### 결과
- Sequence 모두 출력, 위에서 선언된 Classifier가 아래에 붙어 있는 것 확인 가능. 
- num_classes 10으로 수정했기 때문에 그 부분도 수정되어 있음.
- Linear(in_features=4096, out_features=4096) 가 아닌 다른 크기의 이미지를 넣고 싶다면,
    - 위 class VGG에 선언되어 있는 self.classifier 부분의 nn.Linear(512 * 7 * 7, 4096) 부분도 수정해 주어야 함.
- 이 CNN 모델은 동작하지 않을 것
    - custom은 Max Pooling 3번 진행, Max Pooling을 5번 진행한 A, B, D, E와는 size가 안 맞을 것.
    - Max Pooling을 두 번 더 넣어 주거나, FC 부분의 nn.Linear 사이즈를 수정해 주면 정상 동작함.

---

vgg의 기본 코드:
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py

cifar10에 VGG 저장하기

## VGG for cifar10

In [27]:
import torch

In [11]:
import torch.nn as nn

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [24]:
import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

In [25]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [36]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [37]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, # batch_size 512로 바꾸어 줌
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10\cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified


In [39]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
vis.images(images/2 + 0.5)

# show images
#imshow(torchvision.utils.make_grid(images))

# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

 ship horse  bird  frog


vis.image를 띄우고 visdom local host에 접속하면 그림이 뜬다.
</br> batch_size가 512라서 512개의 이미지가 나온다.

## make VGG16 using vgg.py

vgg를 import해서 쓰기 </br>
vgg.py 폴더에다가 바로 넣었기 때문에 import vgg 함

In [50]:
import torchvision.models.vgg as vgg
#import torchvision.models.vgg as vgg

In [51]:
cfg = [32,32,'M', 64,64,128,128,128,'M',256,256,256,512,512,512,'M'] #13 + 3 = vgg16
# convolution의 개수가 13개
# classifier 3개
# 도합 16개

In [52]:
class VGG(nn.Module):

    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        #self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        #AdaptiveAvgPool2d 같은 경우:
            # 내 이미지가 (7,7)보다 크면 상관없음
            # 작게 되면, 당연히 안 쓰는 것이 좋음
            # AdaptiveAvgPool2d -> (7,7)으로 avgerage pooling 맞춰 주는 것
            # 여기서는 굳이 쓸 필요 없다
        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 4096),
            # 4 by 4로 수정
            # 3번의 max pooling 거쳤을 때 나오는 값
            # 32x32 -> 16x16 -> 8x8 -> 4x4
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        #x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [53]:
vgg16= VGG(vgg.make_layers(cfg),10,True).to(device)

In [54]:
a=torch.Tensor(1,3,32,32).to(device)
out = vgg16(a)
print(out)

tensor([[-0.0295,  0.0148, -0.0163,  0.0298, -0.0314, -0.0222,  0.0085,  0.0126,
          0.0215, -0.0167]], grad_fn=<AddmmBackward>)


In [55]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(vgg16.parameters(), lr = 0.005,momentum=0.9)

lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)
# learning rate를 계속 고정시키는 것보다, 감소시키면서 학습시키는 것이 좋을 수도 있음.
# 보다 정밀한 학습을 위해 learning rate를 계속 감소시키도록 lr_schedular 사용함.

# step_size: 다섯 번이 들어가 있는데 아래 스케줄러 스텝을 5번 할 때마다
# 지금 있는 learning rate에 0.9를 곱해라.

## making plot

In [56]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

## training

In [ ]:
print(len(trainloader))
epochs = 50

for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    lr_sche.step()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 30 mini-batches
            loss_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader) ]))
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
        

print('Finished Training')

98


In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
outputs = vgg16(images.to(device))

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:

correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg16(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))